In [1]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import torch
import time
import copy

from grid_env_wrapper import GridEnvWrapper
from grid_env import GridState, GridEnvironment
from dqn import ResNetDQN, GraphDQN  
import torch, torch.nn.functional as F
from ppo import PPOActorCritic
import random

from collections import defaultdict, deque
import heapq
import itertools

import os
import pickle
import datetime
import argparse

In [ ]:
import os
import pickle
import copy
from grid_env_wrapper import GridEnvWrapper  # adjust import as needed


# 2) template env for grid formatting
base_env = GridEnvWrapper(
    grid_size=11,
    variable_grid=False,
    num_dynamic_obstacles=0,
    generation_mode='maze',
    maze_density=0.2
)

def save_datapoint_txts(pkl_path, out_dir, prefix):
    with open(pkl_path, 'rb') as f:
        data_list = pickle.load(f)
    for idx, example in enumerate(data_list):
        agent_envs = example['agent_envs']
        # extract grid
        base_env.env = copy.deepcopy(agent_envs[0].env)
        grid = base_env.env.grid
        rows, cols = grid.shape

        # build lines
        lines = [f"{rows} {cols}"]
        for r in range(rows):
            lines.append(" ".join('@' if grid[r, c] == -1 else '.' for c in range(cols)))
        lines.append(str(len(agent_envs)))
        for env in agent_envs:
            base_env.env = copy.deepcopy(env.env)
            sr, sc = map(int, base_env.env.agent_pos)
            gr, gc = map(int, base_env.env.goal_pos)
            lines.append(f"{sr} {sc} {gr} {gc}")

        # write to file
        fname = f"{prefix}_{idx}.txt"
        with open(os.path.join(out_dir, fname), 'w') as out:
            out.write("\n".join(lines))

# 3) iterate files 5 → 20
for i in range(5, 21):
    pkl_file = os.path.join("simulation_data", f"simulation_datapoints_{i}.pkl")
    if os.path.isfile(pkl_file):
        save_datapoint_txts(pkl_file, "simulation_data/maps", f"map_{i}")
    else:
        print(f"Missing: {pkl_file}")


In [8]:
import os
import pickle
import copy
# Assuming GridEnvWrapper is in the same directory or accessible in PYTHONPATH
# If not, you might need to adjust the import path, e.g., from your_project.grid_env_wrapper import GridEnvWrapper
try:
    from grid_env_wrapper import GridEnvWrapper
except ImportError:
    print("Error: Could not import GridEnvWrapper. Make sure it's in the correct path.")
    print("Please define a placeholder class if you want to proceed with a dummy GridEnvWrapper.")
    # Define a placeholder if the real one is not available for testing script logic
    class GridEnvWrapper:
        def __init__(self, grid_size, **kwargs):
            print(f"Dummy GridEnvWrapper initialized with grid_size: {grid_size}")
            self.grid_size = grid_size
            # Mock a simple env structure for the script to run
            class MockEnv:
                def __init__(self, grid_size_val):
                    self.grid = np.zeros((grid_size_val, grid_size_val)) # Example grid
                    self.agent_pos = (0,0)
                    self.goal_pos = (grid_size_val-1, grid_size_val-1)
            self.env = MockEnv(grid_size)


# Template environment for grid formatting for 21x21 grid
# Adjust maze_density and other parameters if they differ for your 21x21 scenarios
base_env_21x21 = GridEnvWrapper(
    grid_size=25,  # Updated grid size
    # variable_grid=False,
    num_dynamic_obstacles=0,
    generation_mode='warehouse', # Or whatever mode you used for 21x21
    # maze_density=0.2 # Adjust if different for 21x21
)

def save_datapoint_txts(pkl_path, out_dir, prefix, expected_grid_size):
    """
    Saves datapoints from a pickle file to text files in the specified format.

    Args:
        pkl_path (str): Path to the input .pkl file.
        out_dir (str): Directory to save the output .txt files.
        prefix (str): Prefix for the output filenames.
        expected_grid_size (int): The expected grid size (e.g., 11 or 21) for this data.
    """
    try:
        with open(pkl_path, 'rb') as f:
            data_list = pickle.load(f)
    except FileNotFoundError:
        print(f"Error: Pickle file not found at {pkl_path}")
        return
    except Exception as e:
        print(f"Error loading pickle file {pkl_path}: {e}")
        return

    if not os.path.exists(out_dir):
        try:
            os.makedirs(out_dir)
            print(f"Created output directory: {out_dir}")
        except OSError as e:
            print(f"Error creating output directory {out_dir}: {e}")
            return

    for idx, example in enumerate(data_list):
        if 'agent_envs' not in example or not example['agent_envs']:
            print(f"Warning: 'agent_envs' missing or empty in example {idx} from {pkl_path}. Skipping.")
            continue

        # Use a base environment appropriate for the expected grid size
        current_base_env = GridEnvWrapper(grid_size=expected_grid_size) # Re-init for safety or use a passed one
        
        # It's crucial that agent_envs[0].env.grid has the correct dimensions.
        # We'll use the shape from the loaded data for robustness.
        try:
            # Deepcopy to avoid modifying the original loaded data
            temp_env_data = copy.deepcopy(example['agent_envs'][0].env)
            grid = temp_env_data.grid
            rows, cols = grid.shape
        except AttributeError:
            print(f"Warning: Could not access grid or its shape in example {idx} from {pkl_path}. Skipping.")
            continue
        except IndexError:
            print(f"Warning: 'agent_envs' was empty in example {idx} from {pkl_path} after initial check. Skipping.")
            continue


        # Validate grid size if necessary (optional, but good for sanity)
        if rows != expected_grid_size or cols != expected_grid_size:
            print(f"Warning: Mismatch in example {idx} from {pkl_path}. Expected {expected_grid_size}x{expected_grid_size}, got {rows}x{cols}. Proceeding with loaded dimensions.")
            # Or you could choose to skip:
            # print(f"Skipping example {idx} due to grid size mismatch.")
            # continue


        # Build lines for the text file
        lines = [f"{rows} {cols}"]
        for r in range(rows):
            # Assuming -1 represents an obstacle ('@') and other values are traversable ('.')
            lines.append(" ".join('@' if grid[r, c] == -1 else '.' for c in range(cols)))
        
        lines.append(str(len(example['agent_envs'])))
        
        for agent_env_entry in example['agent_envs']:
            try:
                # It's safer to re-assign to a temporary env object or ensure deepcopy
                current_base_env.env = copy.deepcopy(agent_env_entry.env)
                sr, sc = map(int, current_base_env.env.agent_pos)
                gr, gc = map(int, current_base_env.env.goal_pos)
                lines.append(f"{sr} {sc} {gr} {gc}")
            except AttributeError:
                print(f"Warning: Missing agent_pos or goal_pos in an agent_env for example {idx} from {pkl_path}.")
                # Decide how to handle this: skip agent, skip example, or add placeholder
                lines.append("0 0 0 0 # Placeholder for missing agent data") # Example placeholder
            except Exception as e:
                print(f"Warning: Error processing agent data in example {idx} from {pkl_path}: {e}")
                lines.append("0 0 0 0 # Placeholder for error in agent data") # Example placeholder


        # Write to file
        fname = f"{prefix}_{idx}.txt"
        try:
            with open(os.path.join(out_dir, fname), 'w') as out:
                out.write("\n".join(lines))
        except IOError as e:
            print(f"Error writing to file {os.path.join(out_dir, fname)}: {e}")

# Define agent counts for 21x21 grid processing
agent_counts_21x21 = [32, 64, 96]
grid_size_for_these_files = 25
input_data_dir = "simulation_data" # Main directory where simulation_datapoints_AGENTCOUNT.pkl are
output_maps_dir_21x21 = os.path.join("simulation_data", "maps_25x25_2") # Specific output for these maps

# print(f"Processing for 21x21 grid, agent counts: {agent_counts_21x21}")
for num_agents in agent_counts_21x21:
    # Construct the pickle file name based on the agent count
    # This assumes your files are named like "simulation_datapoints_32.pkl", etc.
    pkl_file = os.path.join(input_data_dir, f"simulation_datapoints_2wr_{num_agents}.pkl")
    
    if os.path.isfile(pkl_file):
        print(f"Processing file: {pkl_file}")
        # Define a prefix for the output files, e.g., "map_21x21_32"
        output_prefix = f"map_{num_agents}"
        save_datapoint_txts(pkl_file, output_maps_dir_21x21, output_prefix, grid_size_for_these_files)
    else:
        print(f"Missing datapoint file: {pkl_file}")

# print("\nProcessing for 11x11 grid (original loop) can be added back if needed.")
# Original loop for 5 to 20 agents (11x11 grid)
# print(f"\nProcessing for 11x11 grid, agent counts: 5 to 20")
# grid_size_11x11 = 11
# output_maps_dir_11x11 = os.path.join("simulation_data", "maps_11x11") # Specific output for these maps
# for i in range(5, 21):
#     pkl_file_11x11 = os.path.join(input_data_dir, f"simulation_datapoints_{i}.pkl")
#     if os.path.isfile(pkl_file_11x11):
#         print(f"Processing file: {pkl_file_11x11}")
#         output_prefix_11x11 = f"map_{grid_size_11x11}x{grid_size_11x11}_{i}"
#         save_datapoint_txts(pkl_file_11x11, output_maps_dir_11x11, output_prefix_11x11, grid_size_11x11)
#     else:
#         print(f"Missing datapoint file: {pkl_file_11x11}")

print("\nScript finished.")


Processing file: simulation_data/simulation_datapoints_2wr_32.pkl
Created output directory: simulation_data/maps_25x25_2
Processing file: simulation_data/simulation_datapoints_2wr_64.pkl
Processing file: simulation_data/simulation_datapoints_2wr_96.pkl

Script finished.


In [1]:
import os
import pickle
import copy
# Assuming GridEnvWrapper is in the same directory or accessible in PYTHONPATH
# If not, you might need to adjust the import path, e.g., from your_project.grid_env_wrapper import GridEnvWrapper
try:
    from grid_env_wrapper import GridEnvWrapper
except ImportError:
    print("Error: Could not import GridEnvWrapper. Make sure it's in the correct path.")
    print("Please define a placeholder class if you want to proceed with a dummy GridEnvWrapper.")
    # Define a placeholder if the real one is not available for testing script logic
    class GridEnvWrapper:
        def __init__(self, grid_size, **kwargs):
            print(f"Dummy GridEnvWrapper initialized with grid_size: {grid_size}")
            self.grid_size = grid_size
            # Mock a simple env structure for the script to run
            class MockEnv:
                def __init__(self, grid_size_val):
                    self.grid = np.zeros((grid_size_val, grid_size_val)) # Example grid
                    self.agent_pos = (0,0)
                    self.goal_pos = (grid_size_val-1, grid_size_val-1)
            self.env = MockEnv(grid_size)


# Template environment for grid formatting for 21x21 grid
# Adjust maze_density and other parameters if they differ for your 21x21 scenarios
base_env_21x21 = GridEnvWrapper(
    grid_size=32,  # Updated grid size
    # variable_grid=False,
    num_dynamic_obstacles=0,
    generation_mode='warehouse', # Or whatever mode you used for 21x21
    # maze_density=0.2 # Adjust if different for 21x21
)

def save_datapoint_txts(pkl_path, out_dir, prefix, expected_grid_size):
    """
    Saves datapoints from a pickle file to text files in the specified format.

    Args:
        pkl_path (str): Path to the input .pkl file.
        out_dir (str): Directory to save the output .txt files.
        prefix (str): Prefix for the output filenames.
        expected_grid_size (int): The expected grid size (e.g., 11 or 21) for this data.
    """
    try:
        with open(pkl_path, 'rb') as f:
            data_list = pickle.load(f)
    except FileNotFoundError:
        print(f"Error: Pickle file not found at {pkl_path}")
        return
    except Exception as e:
        print(f"Error loading pickle file {pkl_path}: {e}")
        return

    if not os.path.exists(out_dir):
        try:
            os.makedirs(out_dir)
            print(f"Created output directory: {out_dir}")
        except OSError as e:
            print(f"Error creating output directory {out_dir}: {e}")
            return

    for idx, example in enumerate(data_list):
        if 'agent_envs' not in example or not example['agent_envs']:
            print(f"Warning: 'agent_envs' missing or empty in example {idx} from {pkl_path}. Skipping.")
            continue

        # Use a base environment appropriate for the expected grid size
        current_base_env = GridEnvWrapper(grid_size=expected_grid_size) # Re-init for safety or use a passed one
        
        # It's crucial that agent_envs[0].env.grid has the correct dimensions.
        # We'll use the shape from the loaded data for robustness.
        try:
            # Deepcopy to avoid modifying the original loaded data
            temp_env_data = copy.deepcopy(example['agent_envs'][0].env)
            grid = temp_env_data.grid
            rows, cols = grid.shape
        except AttributeError:
            print(f"Warning: Could not access grid or its shape in example {idx} from {pkl_path}. Skipping.")
            continue
        except IndexError:
            print(f"Warning: 'agent_envs' was empty in example {idx} from {pkl_path} after initial check. Skipping.")
            continue


        # Validate grid size if necessary (optional, but good for sanity)
        if rows != expected_grid_size or cols != expected_grid_size:
            print(f"Warning: Mismatch in example {idx} from {pkl_path}. Expected {expected_grid_size}x{expected_grid_size}, got {rows}x{cols}. Proceeding with loaded dimensions.")
            # Or you could choose to skip:
            # print(f"Skipping example {idx} due to grid size mismatch.")
            # continue


        # Build lines for the text file
        lines = [f"{rows} {cols}"]
        for r in range(rows):
            # Assuming -1 represents an obstacle ('@') and other values are traversable ('.')
            lines.append(" ".join('@' if grid[r, c] == -1 else '.' for c in range(cols)))
        
        lines.append(str(len(example['agent_envs'])))
        
        for agent_env_entry in example['agent_envs']:
            try:
                # It's safer to re-assign to a temporary env object or ensure deepcopy
                current_base_env.env = copy.deepcopy(agent_env_entry.env)
                sr, sc = map(int, current_base_env.env.agent_pos)
                gr, gc = map(int, current_base_env.env.goal_pos)
                lines.append(f"{sr} {sc} {gr} {gc}")
            except AttributeError:
                print(f"Warning: Missing agent_pos or goal_pos in an agent_env for example {idx} from {pkl_path}.")
                # Decide how to handle this: skip agent, skip example, or add placeholder
                lines.append("0 0 0 0 # Placeholder for missing agent data") # Example placeholder
            except Exception as e:
                print(f"Warning: Error processing agent data in example {idx} from {pkl_path}: {e}")
                lines.append("0 0 0 0 # Placeholder for error in agent data") # Example placeholder


        # Write to file
        fname = f"{prefix}_{idx}.txt"
        try:
            with open(os.path.join(out_dir, fname), 'w') as out:
                out.write("\n".join(lines))
        except IOError as e:
            print(f"Error writing to file {os.path.join(out_dir, fname)}: {e}")

# Define agent counts for 21x21 grid processing
agent_counts_21x21 = [8, 16, 32, 64, 96]
grid_size_for_these_files = 32
input_data_dir = "simulation_data" # Main directory where simulation_datapoints_AGENTCOUNT.pkl are
output_maps_dir_21x21 = os.path.join("simulation_data", "den520d_maps") # Specific output for these maps

# print(f"Processing for 21x21 grid, agent counts: {agent_counts_21x21}")
for num_agents in agent_counts_21x21:
    # Construct the pickle file name based on the agent count
    # This assumes your files are named like "simulation_datapoints_32.pkl", etc.
    pkl_file = os.path.join(input_data_dir, f"den520d_{num_agents}.pkl")
    
    if os.path.isfile(pkl_file):
        print(f"Processing file: {pkl_file}")
        # Define a prefix for the output files, e.g., "map_21x21_32"
        output_prefix = f"map_{num_agents}"
        save_datapoint_txts(pkl_file, output_maps_dir_21x21, output_prefix, grid_size_for_these_files)
    else:
        print(f"Missing datapoint file: {pkl_file}")

# print("\nProcessing for 11x11 grid (original loop) can be added back if needed.")
# Original loop for 5 to 20 agents (11x11 grid)
# print(f"\nProcessing for 11x11 grid, agent counts: 5 to 20")
# grid_size_11x11 = 11
# output_maps_dir_11x11 = os.path.join("simulation_data", "maps_11x11") # Specific output for these maps
# for i in range(5, 21):
#     pkl_file_11x11 = os.path.join(input_data_dir, f"simulation_datapoints_{i}.pkl")
#     if os.path.isfile(pkl_file_11x11):
#         print(f"Processing file: {pkl_file_11x11}")
#         output_prefix_11x11 = f"map_{grid_size_11x11}x{grid_size_11x11}_{i}"
#         save_datapoint_txts(pkl_file_11x11, output_maps_dir_11x11, output_prefix_11x11, grid_size_11x11)
#     else:
#         print(f"Missing datapoint file: {pkl_file_11x11}")

print("\nScript finished.")


Processing file: simulation_data/den520d_8.pkl
Created output directory: simulation_data/den520d_maps
Processing file: simulation_data/den520d_16.pkl
Processing file: simulation_data/den520d_32.pkl
Processing file: simulation_data/den520d_64.pkl
Processing file: simulation_data/den520d_96.pkl

Script finished.
